In [21]:
import pandas as pd
import csv
import matplotlib.pyplot as plt


In [22]:
data = pd.read_csv("forestfires.csv")#read data
mapmonth={"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
mapday={"mon":1,"tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [23]:
import numpy as np #add new attribute
data['newarea'] = np.log(1+data["area"])


In [24]:
def getscatterplot(x,y, xlabel):
    plt.scatter(x,y)
    plt.xlabel(xlabel)
    plt.ylabel("newarea")
    

In [25]:
# getscatterplot(data[0], data["area"])
# 2_b_2 plot scatter plot
i = 0
plt.figure(figsize = (20,10))
for cols in data.columns[0:12]:
    plt.subplot(3,4,i+1)
    getscatterplot(data[cols], data["newarea"], cols)
    i+=1
plt.savefig("Scatters.png")
#     print(data[cols])

In [26]:
def pairwiseplot(x, y, xlabel, ylabel):# 2b_3 plot pairwise attributes
    plt.scatter(x,y)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

In [27]:
# this is to plot pairwise scatter plot
checklist= [("X","FFMC"), ("Y", "FFMC"), ("X","temp"),("Y","temp"), ("X", "wind"), ("Y", "wind"), ("wind","rain"), ("temp", "rain"), ("temp", "wind"), ("month", "wind"), ("month", "rain"),("month", "temp"), ("month", "RH"), ("rain", "RH"), ("temp", "RH"), ("wind", "RH")]
plt.figure(figsize = (20,15))
j =0
for a in checklist:
    plt.subplot(4,4, j+1)
    pairwiseplot(data[a[0]], data[a[1]], a[0], a[1])
    j+=1
plt.savefig("pairwise.png")

In [50]:
#2b_4 this is for generatin the calculated result
import csv
with open("chart.csv", "w") as csvfile:
    wt = csv.writer(csvfile)
    wt.writerow(["Variable", "Mean", "Median", "Range", "1st Quantile", "3rd Quantile", "Interquantile Ranges"])
    for c in data.columns:        
        if c == "month":
            mes =[]
            for val in data[c]:
                mes.append(mapmonth[val])
            mes= pd.Series(mes)
            writelist = [c,mes.mean(), mes.median(), (mes.max()-mes.min()), mes.quantile(0.25),mes.quantile(0.75),(mes.quantile(0.75)-mes.quantile(0.25))]
            wt.writerow(writelist)
        elif c =="day":
            mes =[]
            for val in data[c]:
                mes.append(mapday[val])
            mes= pd.Series(mes)
            writelist = [c,mes.mean(), mes.median(), (mes.max()-mes.min()), mes.quantile(0.25),mes.quantile(0.75),(mes.quantile(0.75)-mes.quantile(0.25))]
            wt.writerow(writelist)
        else:
            writelist = [c,data[c].mean(), data[c].median(), (data[c].max()-data[c].min()), data[c].quantile(0.25),data[c].quantile(0.75),(data[c].quantile(0.75)-data[c].quantile(0.25))]
            wt.writerow(writelist)

In [28]:
#2_C this will generate an report for every univariate linear regression 
import statsmodels.formula.api as sma
# singlelist = []
tmp =[]

for c in data.columns[:-2]:
#     print(c)
    if c == "month" or c=='day' or c =="X" or c=="Y":
        res = sma.ols(formula = 'newarea ~ C('+c+')', data = data).fit()
    else:
        res = sma.ols(formula = 'newarea ~ '+c, data = data).fit()
    print(res.summary(),"\n")
#     print(res.params)
    tmp.append(res.params[1:])



                            OLS Regression Results                            
Dep. Variable:                newarea   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.835
Date:                Thu, 01 Feb 2018   Prob (F-statistic):             0.0684
Time:                        11:42:20   Log-Likelihood:                -899.11
No. Observations:                 517   AIC:                             1816.
Df Residuals:                     508   BIC:                             1854.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1520      0.201      5.744      0.0

In [29]:
#plot regression result for quantitative variables in 2c
from sklearn.linear_model import LinearRegression
import numpy as np
plt.figure(figsize = (20,10))
i = 0
coeflist = []
for a in data.columns[4:12]:
    reg = LinearRegression()
#     print(a)
    plt.subplot(2,4,i+1)
    tmpfit = reg.fit(data[a].reshape(-1,1), data["newarea"])
    y_pred = tmpfit.predict(data[a].reshape(-1,1))
    plt.scatter(data[a], data["newarea"])
    plt.plot(data[a], y_pred)
    plt.xlabel(a)
    coeflist.append(reg.coef_[0])
    i+=1
plt.savefig("regressions.png")

C:\Users\hyper\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # This is added back by InteractiveShellApp.init_path()
C:\Users\hyper\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if sys.path[0] == '':


In [30]:
#Saving all univariate coefficient as X
X = []
for v in tmp:
    for k in v:
        X.append(k)
len(X)

39

In [31]:
#2_D multiple variable linear regression
f = pd.DataFrame([[0,0,'aa','aa',0,0,0,0,0,0,0,0,0,0]], columns =data.columns)
data.append(f)
mult = sma.ols(formula='newarea ~ C(X)+C(Y)+C(month)+C(day)+FFMC+DMC+DC+ISI+temp+RH+wind+rain', data = data).fit()
print(mult.summary())
multlist = mult.params

                            OLS Regression Results                            
Dep. Variable:                newarea   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     1.883
Date:                Thu, 01 Feb 2018   Prob (F-statistic):            0.00134
Time:                        11:42:24   Log-Likelihood:                -869.45
No. Observations:                 517   AIC:                             1819.
Df Residuals:                     477   BIC:                             1989.
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.3490      1.645     

In [32]:
#Using all coefficient as Y
Y  = []
for a in multlist[1:]:
    Y.append(a)

In [33]:
#2e now we plot coefficients from 2c and 2d
plt.figure(figsize=(10,5))
plt.xlabel("Univariate")
plt.ylabel("Multiple")
plt.scatter(X,Y)
plt.savefig("effs.png")

In [35]:
#2f X^3 plot
plt.figure(figsize = (20,10))
from scipy.optimize import curve_fit
def func(x, a, b, c,d):
    return a*pow(x,3)+b*pow(x,2)+c*x+d
i=0
for a in data.columns[:-2]:
    if a == "month" or a == 'day' or a =='X' or a == "Y":
        pass
    else:
        plt.subplot(2,4,i+1)
        popt,pcov = curve_fit(func, data[a], data["newarea"])
        plt.scatter(data[a], func(data[a], *popt))
        plt.xlabel(a)
        plt.ylabel("y1")
        plt.scatter(data[a], data["newarea"])
#         print(popt)
        
        i+=1
plt.savefig("x^3.png")

In [36]:
#2f This shows some estimate on cubic regression on all quantitative data with quanlitative variable removed
from statsmodels.api import add_constant
for vals in data.columns[:-2]:
    if vals == "X" or vals =="Y" or vals =="month" or vals == "day":
        pass
    else:
        print(vals)
        x1 = data[vals]
        x2 = np.power(x1,2)
        x3 = np.power(x1,3)
        data["x1"] = x1
        data["x2"] = x2
        data["x3"] = x3
#         print(data.head())
        res = sma.ols(formula='newarea ~ x1 + x2 + x3', data = data).fit()
#         res = model.fit()
        print(res.summary())
        data.drop('x1', axis = 1, inplace = True)
        data.drop('x2', axis = 1, inplace = True)
        data.drop('x3', axis = 1, inplace = True)

C:\Users\hyper\Anaconda3\envs\python3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


FFMC
                            OLS Regression Results                            
Dep. Variable:                newarea   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.4899
Date:                Thu, 01 Feb 2018   Prob (F-statistic):              0.689
Time:                        11:43:35   Log-Likelihood:                -905.73
No. Observations:                 517   AIC:                             1819.
Df Residuals:                     513   BIC:                             1836.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0458      4.208      0.011    

In [37]:
#this check for all interactions between every predictor
# singlelist = []
cnt = 0
savedstr = "newarea ~ "
for c in data.columns[:-2]:
#     print(c)
    if c == "month" or c=='day' or c =="X" or c=="Y":
        cdex = data.columns.get_loc(c)
        for d in data.columns[cdex+1:-2]:
            if d !=c and (d == "month" or d=='day' or d =="X" or d=="Y"):
                mystr = savedstr+"+"+"C("+c+")"+"+"+"C("+d+")"+"+"+"C("+c+")"+"*"+"C("+d+")"
                res = sma.ols(formula = mystr, data = data).fit()
                cnt+=1
                print(res.summary())
            elif d ==c:
                pass
            else:
                mystr = savedstr+"+"+"C("+c+")"+"+"+d+"+"+"C("+c+")"+"*"+d
                res = sma.ols(formula=mystr, data = data).fit()
                cnt+=1
                print(res.summary())
    else:
        cdex = data.columns.get_loc(c)
        for d in data.columns[cdex+1:-2]:
            if d !=c and (d == "month" or d=='day' or d =="X" or d=="Y"):
                mystr = savedstr+c+"+"+"C("+d+")"+"+"+c+"*"+"C("+d+")"
                res = sma.ols(formula = mystr, data = data).fit()
                cnt+=1
                print(res.summary())
            elif d==c:
                pass
            else:
                mystr = savedstr+"+"+c+"+"+"+"+d+"+"+c+"*"+d
                res = sma.ols(formula=mystr, data = data).fit()
                cnt+=1
                print(res.summary())


#     print(res.params)

C:\Users\hyper\Anaconda3\envs\python3\lib\site-packages\statsmodels\base\model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\hyper\Anaconda3\envs\python3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\hyper\Anaconda3\envs\python3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\hyper\Anaconda3\envs\python3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


                            OLS Regression Results                            
Dep. Variable:                newarea   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     1.663
Date:                Thu, 01 Feb 2018   Prob (F-statistic):             0.0113
Time:                        11:43:44   Log-Likelihood:                -876.94
No. Observations:                 517   AIC:                             1826.
Df Residuals:                     481   BIC:                             1979.
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.4346    

                            OLS Regression Results                            
Dep. Variable:                newarea   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.370
Date:                Thu, 01 Feb 2018   Prob (F-statistic):              0.146
Time:                        11:43:45   Log-Likelihood:                -894.67
No. Observations:                 517   AIC:                             1825.
Df Residuals:                     499   BIC:                             1902.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          1.2835      0.464      2.

C:\Users\hyper\Anaconda3\envs\python3\lib\site-packages\statsmodels\regression\linear_model.py:1471: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


                            OLS Regression Results                            
Dep. Variable:                newarea   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     2.280
Date:                Thu, 01 Feb 2018   Prob (F-statistic):            0.00796
Time:                        11:43:45   Log-Likelihood:                -892.80
No. Observations:                 517   AIC:                             1812.
Df Residuals:                     504   BIC:                             1867.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          1.4515      0.465      3.

                            OLS Regression Results                            
Dep. Variable:                newarea   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     1.431
Date:                Thu, 01 Feb 2018   Prob (F-statistic):             0.0933
Time:                        11:43:45   Log-Likelihood:                -890.49
No. Observations:                 517   AIC:                             1827.
Df Residuals:                     494   BIC:                             1925.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -1.2836 

                            OLS Regression Results                            
Dep. Variable:                newarea   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.6156
Date:                Thu, 01 Feb 2018   Prob (F-statistic):              0.605
Time:                        11:43:45   Log-Likelihood:                -905.54
No. Observations:                 517   AIC:                             1819.
Df Residuals:                     513   BIC:                             1836.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0679      1.015      0.067      0.9

                            OLS Regression Results                            
Dep. Variable:                newarea   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.128
Date:                Thu, 01 Feb 2018   Prob (F-statistic):              0.337
Time:                        11:43:46   Log-Likelihood:                -904.77
No. Observations:                 517   AIC:                             1818.
Df Residuals:                     513   BIC:                             1835.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.2910      0.180      7.160      0.0

In [39]:
#2_h fit my own model:
#Y1=DMC+ wind+〖wind〗^2+ 〖wind〗^3+DMC*temp+DC*temp+temp*wind
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
data[data.columns[:-2]], data["newarea"], test_size=0.3, random_state=42)
train = pd.concat([X_train,y_train], axis =1)
train['x1'] = train['wind']
train['x2'] = np.power(train["x1"],2)
train['x3'] = np.power(train["x1"],3)
res = sma.ols(formula='newarea ~ DMC+ wind+x2+x3+DMC*temp+DC*temp+temp*wind', data = train).fit()
train.drop('x1', axis = 1, inplace = True)
train.drop('x2', axis = 1, inplace = True)
train.drop('x3', axis = 1, inplace = True)
res.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                newarea   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1.641
Date:                Thu, 01 Feb 2018   Prob (F-statistic):              0.102
Time:                        11:44:20   Log-Likelihood:                -625.86
No. Observations:                 361   AIC:                             1272.
Df Residuals:                     351   BIC:                             1311.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.7649      1.005     -0.761      0.447      -2.741       1.211
DMC           -0.0073      0.006     -1.172      0.242      -0.020       0.005
wind           0.9902      0.491      2.017      0.044       0.025       1.956
x2            -0.1555      0.111     -1.405      0.161      -0.373       0.062
x3             0.0096      0.008      1.270      0.205      -0.005       0.025
temp           0.0233      0.051      0.453      0.651      -0.078       0.125
DMC:temp       0.0005      0.000      1.535      0.126      -0.000       0.001
DC             0.0013      0.002      0.768      0.443      -0.002       0.005
DC:temp    -5.331e-05   9.67e-05     -0.551      0.582      -0.000       0.000
temp:wind     -0.0132      0.007     -1.791      0.074      -0.028       0.001
==============================================================================
Omnibus:                       62.108   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               89.653
Skew:                           1.163   Prob(JB):                     3.40e-20
Kurtosis:                       3.744   Cond. No.                     1.86e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.86e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [40]:
#2_h testing the model calculate MSE, I did manuall calculatin for RSE
import math
test = pd.concat([X_test,y_test], axis =1)
test['x1'] = test['wind']
test['x2'] = np.power(test["x1"],2)
test['x3'] = np.power(test["x1"],3)
test_pred = res.predict(test)
test.drop('x1', axis = 1, inplace = True)
test.drop('x2', axis = 1, inplace = True)
test.drop('x3', axis = 1, inplace = True)
RSE = math.sqrt(sum(np.power((test_pred-y_test),2))/(len(test)-2))
print (RSE)

1.4007607198270562


In [41]:
#2i-1 knnregression for first 4 variables
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor

le = preprocessing.LabelEncoder()
le.fit(data["month"])
data["newmonth"]=le.transform(data["month"])
le.fit(data["day"])
data["newday"]=le.transform(data["day"])
X = data[["X","Y","newmonth", "newday"]]
x_plotlist = []
yte_plotlist = []
ytr_plotlist = []
# X.head()
for k in range(1,100):
    X_train, X_test, y_train, y_test = train_test_split(
    X, data["newarea"], test_size=0.3, random_state=42)
    neigh = KNeighborsRegressor(n_neighbors = k)
    neigh.fit(X_train, y_train)
    y_pre = neigh.predict(X_test)
    s1=neigh.score(X_test, y_test)
    s2 = neigh.score(X_train, y_train)
    x_plotlist.append(1/k)
    yte_plotlist.append(s1)
    ytr_plotlist.append(s2)
# print(neigh.get_params)
plt.figure(figsize = (20, 10))
plt.subplot(1,2,1)
plt.plot(x_plotlist, yte_plotlist)
plt.xlabel("test 1/k")
plt.ylabel("R^2")
plt.subplot(1,2,2)
plt.plot(x_plotlist, ytr_plotlist)
plt.xlabel("train 1/k")
plt.ylabel("R^2")
plt.savefig("2i1.png")
data.drop('newmonth', axis = 1, inplace = True)
data.drop('newday', axis = 1, inplace = True)
yte_plotlist=[]
ytr_plotlist=[]

In [42]:
#2i-2 knnregression
for k in range(1,100):
    X_train, X_test, y_train, y_test = train_test_split(
    data[data.columns[8:-2]], data["newarea"], test_size=0.3, random_state=42)
    neigh = KNeighborsRegressor(n_neighbors = k)
    neigh.fit(X_train, y_train)
    y_pre = neigh.predict(X_test)
    
    s1=neigh.score(X_test, y_test)
    s2 = neigh.score(X_train, y_train)
    
    yte_plotlist.append(s1)
    ytr_plotlist.append(s2)
plt.figure(figsize = (20, 10))
plt.subplot(1,2,1)
plt.plot(x_plotlist, yte_plotlist)
plt.xlabel("test 1/k")
plt.ylabel("R^2")
plt.subplot(1,2,2)
plt.plot(x_plotlist, ytr_plotlist)
plt.xlabel("train 1/k")
plt.ylabel("R^2")
plt.savefig("2i2.png")
yte_plotlist=[]
ytr_plotlist=[]

In [43]:
#2i-3 knnregression
X = data[["X", "Y","temp", "RH", "wind"]]
for k in range(1,100):
    
    X_train, X_test, y_train, y_test = train_test_split(
    X, data["newarea"], test_size=0.3, random_state=42)
    neigh = KNeighborsRegressor(n_neighbors = k)
    neigh.fit(X_train, y_train)
    y_pre = neigh.predict(X_test)
    s1=neigh.score(X_test, y_test)
    s2 = neigh.score(X_train, y_train)
    
    yte_plotlist.append(s1)
    ytr_plotlist.append(s2)
#     s = neigh.score(X_test, y_test)
#     print(k, s)
plt.figure(figsize = (20, 10))
plt.subplot(1,2,1)
plt.plot(x_plotlist, yte_plotlist)
plt.xlabel("test 1/k")
plt.ylabel("R^2")
plt.subplot(1,2,2)
plt.plot(x_plotlist, ytr_plotlist)
plt.xlabel("train 1/k")
plt.ylabel("R^2")
plt.savefig("2i3.png")
yte_plotlist=[]
ytr_plotlist=[]